# Emporium Logic

In [1]:
import os, sys, random
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import emporium.logic.guild as GD
import emporium.logic.stage as ST

In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Logic Functions

In [8]:
# permited obstacles development

stageType = 'balanced'
maxObstacles = 11

potentialLs = ST.ProductionTable(1,3,1,1,1)
obstacleLs = ST.ObstacleSequence(potentialLs, maxObstacles)

PD.DataFrame(obstacleLs)

,World,Level,Name,Trait,Skill,Success,Failure,Experience,Attack,Damage,Defense,Health,Difficulty
0,1,1,Vanguard,All,Fight,"experience, pass",knockout,13,2.0,8,13.0,11.0,NaN
1,1,1,Sewer Grate,Mig,Tra,"experience, pass next","wound, pass",8,NaN,10,NaN,NaN,14.0
2,1,1,Vanguard,All,Fight,"experience, pass",knockout,13,2.0,8,13.0,11.0,NaN
3,1,1,Gargoyle,Mig,Sab,"experience, pass","wound, pass",8,NaN,10,NaN,NaN,12.0
4,1,1,Spike Trap,Agi,Tra,"experience, pass","wound, pass",8,NaN,10,NaN,NaN,12.0
5,1,1,Vanguard,All,Fight,"experience, pass",knockout,13,2.0,8,13.0,11.0,NaN
6,1,1,Idol,Mig,Per,"treasure, pass",pass,8,NaN,10,NaN,NaN,16.0
7,1,1,Sorcerer,All,Fight,"experience, pass",knockout,13,3.0,10,12.0,9.0,NaN
8,1,1,Chest,Agi,Sab,"treasure, pass",pass,8,NaN,10,NaN,NaN,16.0
9,1,1,Sorcerer,All,Fight,"experience, pass",knockout,13,3.0,10,12.0,9.0,NaN


In [9]:
# permitted = CheckPermitted(obstacleLs, stageType, maxObstacles)

if stageType == 'balanced' and maxObstacles <= 9:
    minDx = {'Agi': 1, 'Cun': 1, 'Mig': 1, 'All': 2}
elif stageType == 'balanced':
    minDx = {'Agi': 2, 'Cun': 2, 'Mig': 2, 'All': 3}
    
if stageType == 'biased agi' and maxObstacles <= 9:
    minDx = {'Agi': 3, 'Cun': 0, 'Mig': 0, 'All': 2}
elif stageType == 'biased agi':
    minDx = {'Agi': 4, 'Cun': 1, 'Mig': 1, 'All': 3}
    
if stageType == 'biased cun' and maxObstacles <= 9:
    minDx = {'Agi': 0, 'Cun': 3, 'Mig': 0, 'All': 2}
elif stageType == 'biased cun':
    minDx = {'Agi': 1, 'Cun': 4, 'Mig': 1, 'All': 3}
    
if stageType == 'biased mig' and maxObstacles <= 9:
    minDx = {'Agi': 0, 'Cun': 0, 'Mig': 3, 'All': 2}
elif stageType == 'biased mig':
    minDx = {'Agi': 1, 'Cun': 1, 'Mig': 4, 'All': 3}

if stageType == 'biased cmb' and maxObstacles <= 9:
    minDx = {'Agi': 1, 'Cun': 1, 'Mig': 1, 'All': 3}
elif stageType == 'biased cmb':
    minDx = {'Agi': 1, 'Cun': 1, 'Mig': 1, 'All': 4}

minDx

{'Agi': 2, 'Cun': 2, 'Mig': 2, 'All': 3}

In [10]:
countDx = {'Agi': 0, 'Cun': 0, 'Mig': 0, 'All': 0}
for ob in obstacleLs:
    countDx[ob['Trait']] += 1

countDx

{'Agi': 2, 'Cun': 1, 'Mig': 3, 'All': 5}

In [11]:
permited = True

if countDx['Agi'] < minDx['Agi'] or countDx['Agi'] > minDx['Agi'] +2:
    permited = False

if countDx['Cun'] < minDx['Cun'] or countDx['Cun'] > minDx['Cun'] +2:
    permited = False

if countDx['Mig'] < minDx['Mig'] or countDx['Mig'] > minDx['Mig'] +2:
    permited = False

if countDx['All'] < minDx['All'] or countDx['All'] > minDx['All'] +2:
    permited = False

permited

False

In [22]:
# run loop to get the number of trials

potentialLs = ST.ProductionTable(1,3,1,1,1)
runs = 1

obstacleLs = ST.ObstacleSequence(potentialLs, maxObstacles)
permitted = ST.CheckPermitted(obstacleLs, stageType, maxObstacles)

while not permitted:
    obstacleLs = ST.ObstacleSequence(potentialLs, maxObstacles)
    permitted = ST.CheckPermitted(obstacleLs, stageType, maxObstacles)
    runs += 1

runs

7